In [1]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install transformers[torch]
!pip install accelerate -U

   ---------------------------------------- 297.3/297.3 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [2]:
import pandas as pd
df=messages = pd.read_csv('J:/IIT Folder/Year_4/FYP/Code/Tests/activity_prompts.csv', sep=',',names=["label", "prompt"])
df.head()

,label,prompt
0,physical,where can I go to do a physical activity
1,physical,Can you recommend something fun and physical f...
2,physical,Could I get a bunch of fun physical activities?
3,physical,I would like to get fit
4,physical,Recommend me some physical activities


In [4]:
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer

model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-base",ignore_mismatched_sizes=True)
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import torch
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer

# Tokenize the prompts
tokenized_prompts = tokenizer(df['prompt'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Create a label-to-integer dictionary
label_to_int = {label: i for i, label in enumerate(df['label'].unique())}

# Convert labels to integers
labels = [label_to_int[label] for label in df['label']]
labels = torch.tensor(labels)
print(labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([0, 0, 0,  ..., 9, 9, 9])


In [6]:
from torch.utils.data import TensorDataset

dataset = TensorDataset(tokenized_prompts['input_ids'], tokenized_prompts['attention_mask'], labels)

In [14]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-05)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        # Forward pass
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    # Evaluation on validation set
    model.eval()
    val_loss = 0
    for batch in val_loader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)
            val_loss += outputs.loss.item()
    val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}')

j:\Code\Python\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
model.save_pretrained('path/to/save/fine-tuned-model')

In [8]:
!pip install transformers[torch]
!pip install accelerate -U

In [9]:
import pandas as pd
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import accelerate
import transformers
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

# Load your data
df = pd.read_csv('J:/IIT Folder/Year_4/FYP/Code/Tests/activity_prompts.csv', sep=',', names=["label", "prompt"])
unique_labels = df['label'].unique()
label_to_int = {label: i for i, label in enumerate(unique_labels)}

# Convert labels to integers
df['label'] = df['label'].map(label_to_int)

# Split your data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize the tokenizer and model
tokenizer = DebertaV2Tokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
model = DebertaV2ForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", num_labels=10, ignore_mismatched_sizes=True)

# Tokenize the data
def tokenize_data(df):
    return tokenizer(df['prompt'].tolist(), padding=True, truncation=True, return_tensors='pt')

train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)
train_labels = torch.tensor(train_df['label'].tolist())
val_labels = torch.tensor(val_df['label'].tolist())

# Create a PyTorch Dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    warmup_steps=500,
    learning_rate=2e-05,
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,  # batch size for evaluation
    warmup_ratio=0.1,  # number of warmup steps for learning rate scheduler
    weight_decay=0.06,  # strength of weight decay
    logging_dir='./logs',  # mixed precision training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('fine_tuned_deberta_model')


# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true=labels, y_pred=predictions, average='macro')

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

Transformers version: 4.36.2
Accelerate version: 0.28.0


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
j:\Code\Python\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
da

{'train_runtime': 1654.2475, 'train_samples_per_second': 2.688, 'train_steps_per_second': 0.085, 'train_loss': 5.347074649325741e+21, 'epoch': 3.0}
